<a href="https://colab.research.google.com/github/IGVF/seqspec/blob/main/examples/seqspec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf seqspec
!git clone https://github.com/igvf/seqspec
!cd seqspec && pip install --quiet .

Cloning into 'seqspec'...
remote: Enumerating objects: 1360, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1360 (delta 220), reused 200 (delta 173), pack-reused 1068
Receiving objects: 100% (1360/1360), 5.22 MiB | 13.42 MiB/s, done.
Resolving deltas: 100% (893/893), done.
  Preparing metadata (setup.py) ... done


In [18]:
!mv seqspec/assays/SPLiT-seq/*onlist* .
!gunzip *onlist*

gzip: barcode-1_onlist.txt: unknown suffix -- ignored
gzip: barcode-2_onlist.txt: unknown suffix -- ignored
gzip: barcode-3_onlist.txt: unknown suffix -- ignored
gzip: index7_onlist.txt: unknown suffix -- ignored


# Interact with an existing spec

In [2]:
!head seqspec/assays/SPLiT-seq/spec.yaml

!Assay
name: SPLiT-seq
doi: https://doi.org/10.1126/science.aam8999
publication_date: 15 March 2018
description: split-pool ligation-based transcriptome sequencing
modalities:
- RNA
lib_struct: https://teichlab.github.io/scg_lib_structs/methods_html/SPLiT-seq.html
assay_spec:
- !Region


In [3]:
!seqspec print seqspec/assays/SPLiT-seq/spec.yaml

                                        ┌─'illumina_p5:29'
                                        │                   ┌─'s5:14'
                                        ├─nextera_read1─────┤
                                        │                   └─'ME1:19'
                                        ├─R1.fastq.gz────── ──'cDNA:98'
                                        │                   ┌─'barcode-1:8'
                                        │                   ├─'linker-2:30'
─────────────────── ──RNA───────────────┤                   ├─'barcode-2:8'
                                        ├─R2.fastq.gz───────┤
                                        │                   ├─'linker-3:30'
                                        │                   ├─'barcode-3:8'
                                        │                   └─'umi:10'
                                        ├─'truseq_read2:34'
                                        ├─I1.fastq.gz────── ──'index7:6'
                    

In [5]:
!seqspec check seqspec/assays/SPLiT-seq/spec.yaml

[error 1] None is not of type 'string' in spec['assay_spec'][0]['regions'][2]['sequence_type']
[error 2] None is not of type 'string' in spec['assay_spec'][0]['regions'][3]['sequence_type']
[error 3] None is not of type 'string' in spec['assay_spec'][0]['regions'][3]['regions'][0]['onlist']['md5']
[error 4] None is not of type 'string' in spec['assay_spec'][0]['regions'][3]['regions'][2]['onlist']['md5']
[error 5] None is not of type 'string' in spec['assay_spec'][0]['regions'][3]['regions'][4]['onlist']['md5']
[error 6] None is not of type 'string' in spec['assay_spec'][0]['regions'][5]['sequence_type']
[error 7] None is not of type 'string' in spec['assay_spec'][0]['regions'][5]['regions'][0]['onlist']['md5']


In [12]:
!seqspec index -m RNA -r R1.fastq.gz seqspec/assays/SPLiT-seq/spec.yaml

R1.fastq.gz	cDNA	0	98


In [13]:
!seqspec index --rev -m RNA -r R2.fastq.gz seqspec/assays/SPLiT-seq/spec.yaml

R2.fastq.gz	umi	0	10
R2.fastq.gz	barcode	10	18
R2.fastq.gz	linker	18	48
R2.fastq.gz	barcode	48	56
R2.fastq.gz	linker	56	86
R2.fastq.gz	barcode	86	94


In [15]:
!seqspec find -m RNA -r barcode-1 seqspec/assays/SPLiT-seq/spec.yaml

- !Region
  region_id: barcode-1
  region_type: barcode
  name: barcode-1
  sequence_type: onlist
  sequence: NNNNNNNN
  min_len: 8
  max_len: 8
  onlist: !Onlist
    filename: barcode-1_onlist.txt
    md5: null
  regions: null
  parent_id: R2.fastq.gz



In [19]:
!seqspec onlist -m RNA -r barcode seqspec/assays/SPLiT-seq/spec.yaml

/content/onlist_joined.txt


In [22]:
!head -2 *onlist*

==> barcode-1_onlist.txt <==
AACGTGAT
AAACATCG

==> barcode-2_onlist.txt <==
AACGTGAT
AAACATCG

==> barcode-3_onlist.txt <==
AACGTGAT
AAACATCG

==> index7_onlist.txt <==
TCGCAG
==> onlist_joined.txt <==
AACGTGATAACGTGATAACGTGAT
AACGTGATAACGTGATAAACATCG


# Create a new spec

In [24]:
!seqspec init -n myassay -m 1 -o spec.yaml "(((barcode:16,umi:12)R1.fastq.gz,(cDNA:150)R2.fastq.gz,(index7:8)I1.fastq.gz)rna)"

In [25]:
!seqspec print spec.yaml

                                             ┌─'barcode:16'
                              ┌─R1.fastq.gz──┤
                              │              └─'umi:12'
────────────── ──rna──────────┤
                              ├─R2.fastq.gz─ ──'cDNA:150'
                              └─I1.fastq.gz─ ──'index7:8'


In [26]:
!seqspec format -o fmt.yaml spec.yaml

In [27]:
!seqspec check fmt.yaml